<a href="https://colab.research.google.com/github/OliverRevilla/Machine-Learning-Python/blob/main/Tree_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import confusion_matrix, plot_confusion_matrix,accuracy_score
from sklearn.model_selection import cross_val_score

In [6]:
link = "/content/drive/MyDrive/Machine Learning Python/CURSO INFOPUC/Controles/Copia de sensor lb.csv"
dataset = pd.read_csv(link, sep = ";")
dataset.head()

,Activo_id,ciclo,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,max_ciclo,count_down
0,1,1,518.67,641.82,1589.70,1400.60,14.62,21.61,554.36,2388.06,9046.19,1.3,47.47,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100,39.06,23.4190,192,191
1,1,2,518.67,642.15,1591.82,1403.14,14.62,21.61,553.75,2388.04,9044.07,1.3,47.49,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100,39.00,23.4236,192,190
2,1,3,518.67,642.35,1587.99,1404.20,14.62,21.61,554.26,2388.08,9052.94,1.3,47.27,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100,38.95,23.3442,192,189
3,1,4,518.67,642.35,1582.79,1401.87,14.62,21.61,554.45,2388.11,9049.48,1.3,47.13,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100,38.88,23.3739,192,188
4,1,5,518.67,642.37,1582.85,1406.22,14.62,21.61,554.00,2388.06,9055.15,1.3,47.28,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100,38.90,23.4044,192,187


In [8]:
dataset['vida'] = dataset.ciclo / dataset.max_ciclo
dataset['target'] = np.where(dataset.vida < 0.8,0,1)
dataset.head()

,Activo_id,ciclo,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,max_ciclo,count_down,vida,target
0,1,1,518.67,641.82,1589.70,1400.60,14.62,21.61,554.36,2388.06,9046.19,1.3,47.47,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100,39.06,23.4190,192,191,0.005208,0
1,1,2,518.67,642.15,1591.82,1403.14,14.62,21.61,553.75,2388.04,9044.07,1.3,47.49,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100,39.00,23.4236,192,190,0.010417,0
2,1,3,518.67,642.35,1587.99,1404.20,14.62,21.61,554.26,2388.08,9052.94,1.3,47.27,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100,38.95,23.3442,192,189,0.015625,0
3,1,4,518.67,642.35,1582.79,1401.87,14.62,21.61,554.45,2388.11,9049.48,1.3,47.13,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100,38.88,23.3739,192,188,0.020833,0
4,1,5,518.67,642.37,1582.85,1406.22,14.62,21.61,554.00,2388.06,9055.15,1.3,47.28,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100,38.90,23.4044,192,187,0.026042,0


In [9]:
col_elim = ['vida','max_ciclo','count_down']

dataset = dataset.drop(columns = col_elim, axis = 1)
dataset.head()

,Activo_id,ciclo,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,target
0,1,1,518.67,641.82,1589.70,1400.60,14.62,21.61,554.36,2388.06,9046.19,1.3,47.47,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100,39.06,23.4190,0
1,1,2,518.67,642.15,1591.82,1403.14,14.62,21.61,553.75,2388.04,9044.07,1.3,47.49,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100,39.00,23.4236,0
2,1,3,518.67,642.35,1587.99,1404.20,14.62,21.61,554.26,2388.08,9052.94,1.3,47.27,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100,38.95,23.3442,0
3,1,4,518.67,642.35,1582.79,1401.87,14.62,21.61,554.45,2388.11,9049.48,1.3,47.13,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100,38.88,23.3739,0
4,1,5,518.67,642.37,1582.85,1406.22,14.62,21.61,554.00,2388.06,9055.15,1.3,47.28,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100,38.90,23.4044,0


In [12]:
dataset = dataset.query("Activo_id <= 20")

Matriz de características

In [21]:
X = dataset.iloc[:,1:-1].values
Y = dataset.iloc[:,-1].values

Dividir entre conjunto de entrenamiento y de evaluación

In [23]:

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 0)

Creación del modelo de árbol de regresión

In [24]:

modelo_regresion = DecisionTreeRegressor(random_state = 0, max_leaf_nodes = 10)
modelo_regresion.fit(X_train, Y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=10,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=0, splitter='best')

In [26]:
# Realizando las predicciones correspondientes
y_pred = modelo_regresion.predict(X_test)
y.shape

(834,)

In [27]:
# Reealizando la comparación entre la predición y el modelo
comparacion = np.concatenate([Y_test.reshape(-1,1), y_pred.reshape(-1,1)], axis = 1)
matriz_comparacion = pd.DataFrame(comparacion, columns = ['Real','Modelo'])
matriz_comparacion.head()

,Real,Modelo
0,0.0,0.021739
1,0.0,0.014452
2,1.0,0.705426
3,0.0,0.014452
4,0.0,0.014452


In [28]:
# Discretizando las variables

matriz_comparacion['Y_pred'] = np.where(matriz_comparacion['Modelo'] > 0.5,1,0)
matriz_comparacion.head()

,Real,Modelo,Y_pred
0,0.0,0.021739,0
1,0.0,0.014452,0
2,1.0,0.705426,1
3,0.0,0.014452,0
4,0.0,0.014452,0


In [31]:

y_pred = matriz_comparacion.Y_pred

In [33]:
matriz_confusion = confusion_matrix(Y_test,y_pred)
matriz_confusion

array([[633,  28],
       [ 11, 162]])

In [35]:
accuracy_score(Y_test,y_pred)

0.9532374100719424

Aplicando K-Fold Cross Validation

In [36]:

accuracies = cross_val_score(modelo_regresion,X_train,Y_train,cv = 10)
print(accuracies.mean())
print(accuracies.std())

0.7718132502206425
0.0462626393623578
